In [227]:
import pandas as pd

In [228]:
df = pd.read_csv("Retail_Sample.csv")

In [229]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   LOAN_ID                20 non-null     int64 
 1   REGION_NAME            20 non-null     object
 2   PRINCIPAL_TOTAL        20 non-null     int64 
 3   DISBURSEMENT_DATE      20 non-null     object
 4   PRINCIPAL_OUTSTANDING  20 non-null     int64 
 5   ZONE_ID                20 non-null     object
 6   SALE_AGENT             20 non-null     object
 7   SOCIAL_SCORE           20 non-null     int64 
 8   State                  20 non-null     object
 9   Branch                 20 non-null     object
 10  INDUSTRY               20 non-null     object
dtypes: int64(4), object(7)
memory usage: 1.8+ KB


In [230]:
df.head(2)

,LOAN_ID,REGION_NAME,PRINCIPAL_TOTAL,DISBURSEMENT_DATE,PRINCIPAL_OUTSTANDING,ZONE_ID,SALE_AGENT,SOCIAL_SCORE,State,Branch,INDUSTRY
0,1,GUJARAT,314175,31-07-20,10260,WEST,90053,5,GUJARAT,AHMEDABAD,DAIRY
1,2,PUNJAB,364390,31-07-20,22864,NORTH,90150,6,PUNJAB,LUDHIANA,KIRANA


In [231]:
# Define pivot components
rows_list = ['ZONE_ID','REGION_NAME']         # Rows to group by
columns_list = ['State']           # Columns to pivot
filters_list = {}#{'SALE_AGENT':['90053','90150','AG11100','AG11101', '90125']}    
values_list = {'PRINCIPAL_TOTAL','SOCIAL_SCORE'}# Filters (e.g., {'Value1': 30})
aggfunc_list = {'PRINCIPAL_TOTAL': ['sum'],'SOCIAL_SCORE':['min']}  # Aggregation functions


In [263]:
# Define pivot components
rows_list = ['ZONE_ID','REGION_NAME','State']         # Rows to group by
columns_list = []           # Columns to pivot
filters_list = {}#{'SALE_AGENT':['90053','90150','AG11100','AG11101', '90125']}    
values_list = {'PRINCIPAL_TOTAL','PRINCIPAL_OUTSTANDING'}# Filters (e.g., {'Value1': 30})
aggfunc_list = {'PRINCIPAL_TOTAL': ['sum'],'PRINCIPAL_OUTSTANDING':['sum']}  # Aggregation functions

In [264]:
if filters_list:
    for col, val in filters_list.items():
        df = df[df[col].apply(lambda x: any(item in x for item in val))]

grouped = df.groupby(rows_list + columns_list).agg(aggfunc_list)
pivot = grouped.unstack(columns_list, fill_value=0)

## col wise total...
for total in values_list:
    new_df = pivot[total]
    pivot['Total of ' + total] = new_df.sum(axis=1)

# row wise total
new_row = []
for col in pivot.columns:
    new_row.append(pivot[col].sum())
new_row_df = pd.DataFrame([new_row], columns=pivot.columns)
result = pd.concat([pivot, new_row_df])

#### final..
print(result)
# result.to_csv('test.csv')
print(pivot)

                                       PRINCIPAL_TOTAL PRINCIPAL_OUTSTANDING  \
                                                   sum                   sum   
(EAST, WEST_BENGAL, WEST_BENGAL)                362145                 34366   
(NORTH, HARYANA, DELHI/NCR)                     315144                 62961   
(NORTH, HARYANA, HARYANA)                      2539982               1592428   
(NORTH, PUNJAB, PUNJAB)                         364390                 22864   
(WEST, GUJARAT, GUJARAT)                       1789360                135671   
(WEST, MADHYA_PRADESH, MADHYA_PRADESH)         1090615                441547   
(WEST, MAHARASHTRA, MAHARASHTRA)                721636                252814   
0                                              7183272               2542651   

                                       Total of PRINCIPAL_OUTSTANDING  \
                                                                        
(EAST, WEST_BENGAL, WEST_BENGAL)                     

In [266]:
idx = pd.IndexSlice
pivot.loc[idx['WEST', :], :]

PRINCIPAL_TOTAL PRINCIPAL_OUTSTANDING  \
                                                  sum                   sum   
ZONE_ID REGION_NAME    State                                                  
WEST    GUJARAT        GUJARAT                1789360                135671   
        MADHYA_PRADESH MADHYA_PRADESH         1090615                441547   
        MAHARASHTRA    MAHARASHTRA             721636                252814   

                                      Total of PRINCIPAL_OUTSTANDING  \
                                                                       
ZONE_ID REGION_NAME    State                                           
WEST    GUJARAT        GUJARAT                                135671   
        MADHYA_PRADESH MADHYA_PRADESH                         441547   
        MAHARASHTRA    MAHARASHTRA                            252814   

                                      Total of PRINCIPAL_TOTAL  
                                                                
ZONE_ID REGION_NAME    State                                    
WEST    GUJARAT        GUJARAT                         1789360  
        MADHYA_PRADESH MADHYA_PRADESH                  1090615  
        MAHARASHTRA    MAHARASHTRA                      721636

In [267]:
# n = len(result.index[0])
result = result[:-1,:]

for i in range(n):
    lst = list(set(pivot.index.levels[i]))
    print(lst)
    for l in lst:
        result.loc[l]

InvalidIndexError: (slice(None, -1, None), slice(None, None, None))

In [268]:
########### fnal code
if filters_list:
    for col, val in filters_list.items():
        df = df[df[col].apply(lambda x: any(item in x for item in val))]

grouped = df.groupby(rows_list + columns_list).agg(aggfunc_list)
pivot = grouped.unstack(columns_list, fill_value=0)

# ## col wise total...
# for total in values_list:
#     new_df = pivot[total]
#     pivot['Total of ' + total] = new_df.sum(axis=1)

In [269]:
pivot

PRINCIPAL_TOTAL PRINCIPAL_OUTSTANDING
                                                  sum                   sum
ZONE_ID REGION_NAME    State                                               
EAST    WEST_BENGAL    WEST_BENGAL             362145                 34366
NORTH   HARYANA        DELHI/NCR               315144                 62961
                       HARYANA                2539982               1592428
        PUNJAB         PUNJAB                  364390                 22864
WEST    GUJARAT        GUJARAT                1789360                135671
        MADHYA_PRADESH MADHYA_PRADESH         1090615                441547
        MAHARASHTRA    MAHARASHTRA             721636                252814

In [274]:
n = len(pivot.index[0])
idx = pd.IndexSlice
for i in range(n-1):# size of index
    lst = list(set(pivot.index.levels[i])) #ith level of index
    # print(lst)
    for l in lst:# values of indexlevel
        new_row = []
        child_df = pivot.loc[idx[l, :], :]
        # agg for ith level of index...
        for col in child_df.columns:
            new_row.append(child_df[col].sum())
        new_row_df = pd.DataFrame([new_row], columns=child_df.columns)
        child_df_new = pd.concat([new_row_df, child_df])
        print(child_df_new)
        # print(dd)

                            PRINCIPAL_TOTAL PRINCIPAL_OUTSTANDING
                                        sum                   sum
0                                   3219516               1678253
(NORTH, HARYANA, DELHI/NCR)          315144                 62961
(NORTH, HARYANA, HARYANA)           2539982               1592428
(NORTH, PUNJAB, PUNJAB)              364390                 22864
                                 PRINCIPAL_TOTAL PRINCIPAL_OUTSTANDING
                                             sum                   sum
0                                         362145                 34366
(EAST, WEST_BENGAL, WEST_BENGAL)          362145                 34366
                                       PRINCIPAL_TOTAL PRINCIPAL_OUTSTANDING
                                                   sum                   sum
0                                              3601611                830032
(WEST, GUJARAT, GUJARAT)                       1789360                135671
(WEST, MADHY

KeyError: 'MADHYA_PRADESH'

In [307]:
n = len(pivot.index[0])
idx = pd.IndexSlice
for i in range(n):# size of index
    lst = list(set(pivot.index.levels[i])) #ith level of index
    print(f"level--------{i}")
    print(lst)
    for lst_ele in lst:# values of indexlevel
        new_row = []
        arr = [slice(None) if ind != i else lst_ele for ind in range(n)]
        # print(arr)
        idx = pd.IndexSlice[tuple(arr)]
        child_df = pivot.loc[idx, :]
        # print(child_df)
        # agg for ith level of index...
        for col in child_df.columns:
            new_row.append(child_df[col].sum())
        new_row_df = pd.DataFrame([new_row], columns=child_df.columns, index=[lst_ele])
        child_df_new = pd.concat([new_row_df, child_df])
        print(child_df_new)
    print("********************************")

level--------0
['NORTH', 'EAST', 'WEST']
                            PRINCIPAL_TOTAL PRINCIPAL_OUTSTANDING
                                        sum                   sum
NORTH                               3219516               1678253
(NORTH, HARYANA, DELHI/NCR)          315144                 62961
(NORTH, HARYANA, HARYANA)           2539982               1592428
(NORTH, PUNJAB, PUNJAB)              364390                 22864
                                 PRINCIPAL_TOTAL PRINCIPAL_OUTSTANDING
                                             sum                   sum
EAST                                      362145                 34366
(EAST, WEST_BENGAL, WEST_BENGAL)          362145                 34366
                                       PRINCIPAL_TOTAL PRINCIPAL_OUTSTANDING
                                                   sum                   sum
WEST                                           3601611                830032
(WEST, GUJARAT, GUJARAT)                       1

In [291]:
# Parameters
n = 3  # Number of levels in MultiIndex
i = 1  # Level we want to fix (State level)
l = 'HARYANA'  # Value to match for the State level

# Generate the IndexSlice dynamically
arr = [slice(None) if ind != i else l for ind in range(n)]
print("Generated Slice:", arr)

# Use the generated slice with pd.IndexSlice
idx = pd.IndexSlice[tuple(arr)]

# Access the data
result = pivot.loc[idx, :]
print("\nFiltered Data:")
print(result)


Generated Slice: [slice(None, None, None), 'HARYANA', slice(None, None, None)]

Filtered Data:
                              PRINCIPAL_TOTAL PRINCIPAL_OUTSTANDING
                                          sum                   sum
ZONE_ID REGION_NAME State                                          
NORTH   HARYANA     DELHI/NCR          315144                 62961
                    HARYANA           2539982               1592428
